In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import imdb
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Conv2DTranspose,MaxPooling1D, Embedding,Input, Conv1D, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from tensorflow.keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils
from keras.preprocessing import sequence

In [3]:
top_words = 1000
# Now we split our data-set into training and test data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


model = Sequential()
model.add(Embedding(1000, 32, input_length=max_words))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu',name="d1"))

model.add(Dense(1, activation='sigmoid', name="d2"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           32000     
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           6208      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2048128   
_________________________________________________________________
d1 (Dense)                   (None, 256)               3

In [5]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
196/196 [==============================] - 35s 29ms/step - loss: 0.4759 - accuracy: 0.7401 - val_loss: 0.3147 - val_accuracy: 0.8632
Epoch 2/10
196/196 [==============================] - 5s 27ms/step - loss: 0.2950 - accuracy: 0.8761 - val_loss: 0.2968 - val_accuracy: 0.8728
Epoch 3/10
196/196 [==============================] - 5s 27ms/step - loss: 0.2616 - accuracy: 0.8913 - val_loss: 0.2835 - val_accuracy: 0.8801
Epoch 4/10
196/196 [==============================] - 5s 27ms/step - loss: 0.2381 - accuracy: 0.9014 - val_loss: 0.2844 - val_accuracy: 0.8801
Epoch 5/10
196/196 [==============================] - 5s 27ms/step - loss: 0.1915 - accuracy: 0.9213 - val_loss: 0.3082 - val_accuracy: 0.8732
Epoch 6/10
196/196 [==============================] - 5s 27ms/step - loss: 0.1467 - accuracy: 0.9437 - val_loss: 0.3386 - val_accuracy: 0.8738
Epoch 7/10
196/196 [==============================] - 5s 27ms/step - loss: 0.0907 - accuracy: 0.9660 - val_loss: 0.5135 - val_accuracy: 0.852

In [6]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 4s 5ms/step - loss: 0.7121 - accuracy: 0.8551
Accuracy: 85.51%


In [7]:
teacher_WO_Softmax = Model(model.input, model.get_layer('d1').output)
train_dense = teacher_WO_Softmax.predict(X_train)

s1Train=train_dense[:,:64]
s2Train=train_dense[:,64:128]
s3Train=train_dense[:,128:192]
s4Train=train_dense[:,192:]


In [8]:

def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=(500,))
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile('adam',loss=['binary_crossentropy','mse'],loss_weights=[0.5,0.5],metrics=['accuracy'])

    return gan

In [9]:
def build_sdiscriminator():
    
    input2 = Input(shape=(64,),name='input')
    inp=Dense(128)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    conv3 = Dense(256,activation='relu')(leaky_relu)
    b_n = BatchNormalization()(conv3)
    b_n=Dropout(0.25)(b_n)
    conv3 = Dense(256,activation='relu')(b_n)
    b_n = BatchNormalization()(conv3)
    conv4 = Dense(512,activation='relu')(b_n)
    b_n = BatchNormalization()(conv4)
    dense = Dense(1,activation='sigmoid')(b_n)
    output2=Dense(64,activation='relu')(b_n)
    disc = Model(input2,[dense,output2])          
    disc.compile(Adam(lr=0.0002,beta_1=0.5),loss=['binary_crossentropy','mse',],metrics=['accuracy'])


    return disc

In [21]:
def define_model(name):
  model = Sequential()
  model.add(tf.keras.layers.Embedding(1000, 10, input_length=500,name=name))
  model.add(Dropout(0.2))
  model.add(Conv1D(128, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())
  model.add(Conv1D(32, 3,strides=3,
                  activation='relu'))
  model.add(MaxPooling1D())

  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu',name=name+"req"))
	# compile model
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  return model

In [22]:
define_model("s1").summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Embedding)               (None, 500, 10)           10000     
_________________________________________________________________
dropout_15 (Dropout)         (None, 500, 10)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 166, 128)          3968      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 83, 128)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 27, 32)            12320     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 13, 32)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 416)              

In [23]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    # discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
            
       
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [24]:
s1=define_model("s1")
d1 = build_sdiscriminator()
gan1 = build_gan(s1,d1)

s2=define_model("s2")
d2 = build_sdiscriminator()
gan2 = build_gan(s2,d2)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [25]:
s3=define_model("s3")
d3 = build_sdiscriminator()
gan3 = build_gan(s3,d3)

s4=define_model("s4")
d4 = build_sdiscriminator()
gan4 = build_gan(s4,d4)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [26]:
s1=training(s1,d1,gan1,s1Train,20)
s2=training(s2,d2,gan2,s2Train,20)

195/195 [==============================] - 14s 59ms/step
Epoch: 1 | Discriminator Loss: 0.937800 | Generator Loss: 0.382060 | 
195/195 [==============================] - 12s 59ms/step
Epoch: 2 | Discriminator Loss: 0.813715 | Generator Loss: 0.314670 | 
195/195 [==============================] - 12s 59ms/step
Epoch: 3 | Discriminator Loss: 0.800788 | Generator Loss: 0.340879 | 
195/195 [==============================] - 11s 59ms/step
Epoch: 4 | Discriminator Loss: 0.787773 | Generator Loss: 0.351576 | 
195/195 [==============================] - 12s 59ms/step
Epoch: 5 | Discriminator Loss: 0.773963 | Generator Loss: 0.354105 | 
195/195 [==============================] - 12s 59ms/step
Epoch: 6 | Discriminator Loss: 0.768029 | Generator Loss: 0.376178 | 
195/195 [==============================] - 12s 61ms/step
Epoch: 7 | Discriminator Loss: 0.768840 | Generator Loss: 0.383924 | 
195/195 [==============================] - 12s 61ms/step
Epoch: 8 | Discriminator Loss: 0.787347 | Generator Lo

In [27]:
s3=training(s3,d3,gan3,s3Train,20)
s4=training(s4,d4,gan4,s4Train,20)

195/195 [==============================] - 14s 58ms/step
Epoch: 1 | Discriminator Loss: 0.955467 | Generator Loss: 0.372942 | 
195/195 [==============================] - 11s 59ms/step
Epoch: 2 | Discriminator Loss: 0.812343 | Generator Loss: 0.331212 | 
195/195 [==============================] - 11s 58ms/step
Epoch: 3 | Discriminator Loss: 0.776324 | Generator Loss: 0.311328 | 
195/195 [==============================] - 11s 58ms/step
Epoch: 4 | Discriminator Loss: 0.767966 | Generator Loss: 0.344622 | 
195/195 [==============================] - 11s 59ms/step
Epoch: 5 | Discriminator Loss: 0.763996 | Generator Loss: 0.320691 | 
195/195 [==============================] - 11s 58ms/step
Epoch: 6 | Discriminator Loss: 0.762308 | Generator Loss: 0.355237 | 
195/195 [==============================] - 12s 62ms/step
Epoch: 7 | Discriminator Loss: 0.761813 | Generator Loss: 0.352255 | 
195/195 [==============================] - 12s 59ms/step
Epoch: 8 | Discriminator Loss: 0.756711 | Generator Lo

In [28]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output
o3=s3.get_layer("s3req").output
o4=s4.get_layer("s4req").output

out = tensorflow.keras.layers.concatenate([o1,o2,o3,o4])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)

mm2=Model([s1.get_layer("s1").input,
           s2.get_layer("s2").input,
           s3.get_layer("s3").input,
           s4.get_layer("s4").input],
          output3)
my_weights=model.get_layer('d2').get_weights()
mm2.get_layer('d11').set_weights(my_weights)

In [29]:
i=0
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

In [30]:
adam = Adam(0.0002,0.5,0.999)

mm2.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [31]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train,X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test,X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 6s 23ms/step - loss: 1.3879 - accuracy: 0.5796 - val_loss: 0.6047 - val_accuracy: 0.7348
Epoch 2/10
196/196 [==============================] - 4s 21ms/step - loss: 0.9721 - accuracy: 0.6540 - val_loss: 0.4881 - val_accuracy: 0.7971
Epoch 3/10
196/196 [==============================] - 4s 21ms/step - loss: 0.7953 - accuracy: 0.7032 - val_loss: 0.4432 - val_accuracy: 0.8172
Epoch 4/10
196/196 [==============================] - 4s 21ms/step - loss: 0.6749 - accuracy: 0.7381 - val_loss: 0.4326 - val_accuracy: 0.8242
Epoch 5/10
196/196 [==============================] - 4s 21ms/step - loss: 0.6044 - accuracy: 0.7666 - val_loss: 0.4201 - val_accuracy: 0.8289
Epoch 6/10
196/196 [==============================] - 4s 21ms/step - loss: 0.5600 - accuracy: 0.7830 - val_loss: 0.4124 - val_accuracy: 0.8318
Epoch 7/10
196/196 [==============================] - 4s 21ms/step - loss: 0.5160 - accuracy: 0.7962 - val_loss: 0.4064 - val_accuracy: 0.8338

In [32]:
# 1 student evaluate model
_, acc = mm2.evaluate([X_test,X_test,X_test,X_test], y_test, verbose=1)
print('> %.2f' % (acc * 100.0))

782/782 [==============================] - 6s 7ms/step - loss: 0.3900 - accuracy: 0.8355
> 83.55


In [33]:
mm2.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 500)]        0                                            
____________________________________________________________________________________________

## Feed Forward Method


In [34]:
s1= define_model("s1")
s2= define_model("s2")
s3= define_model("s3")
s4= define_model("s4")

In [35]:
batch_size = 128
mm_history=s1.fit(X_train, s1Train, batch_size=batch_size, epochs=10,verbose=1)
mm2_history=s2.fit(X_train, s2Train, batch_size=batch_size, epochs=10,verbose=1)


Epoch 1/10
196/196 [==============================] - 3s 11ms/step - loss: 0.1837 - accuracy: 0.1374
Epoch 2/10
196/196 [==============================] - 2s 10ms/step - loss: 0.1050 - accuracy: 0.2814
Epoch 3/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0820 - accuracy: 0.3706
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0765 - accuracy: 0.3993
Epoch 5/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0749 - accuracy: 0.4315
Epoch 6/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0716 - accuracy: 0.4529
Epoch 7/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0706 - accuracy: 0.4712
Epoch 8/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0691 - accuracy: 0.4798
Epoch 9/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0678 - accuracy: 0.4921
Epoch 10/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0676 - accura

In [36]:
mm3_history=s3.fit(X_train, s3Train, batch_size=batch_size, epochs=10,verbose=1)
mm4_history=s4.fit(X_train, s4Train, batch_size=batch_size, epochs=10,verbose=1)

Epoch 1/10
196/196 [==============================] - 3s 11ms/step - loss: 0.1981 - accuracy: 0.1624
Epoch 2/10
196/196 [==============================] - 2s 11ms/step - loss: 0.1164 - accuracy: 0.3222
Epoch 3/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0925 - accuracy: 0.4004
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0871 - accuracy: 0.4301
Epoch 5/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0834 - accuracy: 0.4603
Epoch 6/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0800 - accuracy: 0.4752
Epoch 7/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0777 - accuracy: 0.4962
Epoch 8/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0774 - accuracy: 0.5032
Epoch 9/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0761 - accuracy: 0.5094
Epoch 10/10
196/196 [==============================] - 2s 11ms/step - loss: 0.0751 - accura

In [37]:
o1=s1.get_layer("s1req").output
o2=s2.get_layer("s2req").output
o3=s3.get_layer("s3req").output
o4=s4.get_layer("s4req").output
out = tensorflow.keras.layers.concatenate([o1,o2,o3,o4])

output=Activation('relu')(out)
output2=Dropout(0.5)(output)
output3=Dense(1,activation="sigmoid", name="d11")(output2)
# output4=Activation('softmax')(output3)
mm=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,
          s3.get_layer("s3").input,s4.get_layer("s4").input], 
         output3)
my_weights=model.get_layer('d2').get_weights()
mm.get_layer('d11').set_weights(my_weights)

In [38]:
i=0
for l in mm.layers[:len(mm.layers)-1]:
    l.trainable=False

In [39]:
adam = Adam(0.0002,0.5,0.999)

mm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
batch_size = 128
mm_history=mm.fit([X_train,X_train,X_train,X_train], y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_test,X_test,X_test,X_test], y_test))

Epoch 1/10
196/196 [==============================] - 7s 26ms/step - loss: 0.4519 - accuracy: 0.8613 - val_loss: 0.4180 - val_accuracy: 0.8238
Epoch 2/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3357 - accuracy: 0.8559 - val_loss: 0.4000 - val_accuracy: 0.8218
Epoch 3/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3319 - accuracy: 0.8598 - val_loss: 0.3963 - val_accuracy: 0.8230
Epoch 4/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3280 - accuracy: 0.8608 - val_loss: 0.3951 - val_accuracy: 0.8234
Epoch 5/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3275 - accuracy: 0.8601 - val_loss: 0.3871 - val_accuracy: 0.8255
Epoch 6/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3251 - accuracy: 0.8610 - val_loss: 0.3908 - val_accuracy: 0.8239
Epoch 7/10
196/196 [==============================] - 4s 23ms/step - loss: 0.3268 - accuracy: 0.8597 - val_loss: 0.3928 - val_accuracy: 0.8233

In [41]:
_, acc = mm.evaluate([X_test,X_test,X_test,X_test], y_test, verbose=1)

782/782 [==============================] - 6s 8ms/step - loss: 0.3899 - accuracy: 0.8241


In [42]:
mm.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 500)]        0                                            
___________________________________________________________________________________________